# Chapter 3. Tensors

## Imports

In [57]:
import torch
import numpy as np

## Tensor introduction

In [2]:
a = torch.ones(3)

In [3]:
float(a[1])

1.0

In [4]:
a[2] = 2.0

In [5]:
a

tensor([1., 1., 2.])

In [6]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [7]:
points = torch.tensor([[4, 1], [5, 3], [2, 1]], dtype=torch.float32)
points, points.dtype

(tensor([[4., 1.],
         [5., 3.],
         [2., 1.]]),
 torch.float32)

In [8]:
points[1]

tensor([5., 3.])

In [9]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [10]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [11]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [12]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
unsqueezed_weights.shape

torch.Size([3, 1, 1])

In [13]:
img_weights  = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights

img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

## Named tensors

In [14]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=["channels"])
weights_named

/tmp/ipykernel_379718/2918111397.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1758.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=["channels"])


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [15]:
img_named = img_t.refine_names(..., "channels", "rows", "columns")
print("img_named:", img_named.shape, img_named.names)

img_named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')


In [16]:
batch_named = batch_t.refine_names(..., "channels", "rows", "columns")
print("batch_named:", batch_named.shape, batch_named.names)

batch_named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [17]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [18]:
weights_aligned_batch = weights_named.align_as(batch_named)
weights_aligned_batch.shape, weights_aligned_batch.names

(torch.Size([1, 3, 1, 1]), (None, 'channels', 'rows', 'columns'))

In [19]:
gray_named = (img_named * weights_aligned).sum("channels")
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [21]:
# error
# gray_named = (img_named * weights_named).sum("channels")

In [22]:
gray_plain = gray_named.rename(None)

In [23]:
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

## Tensor element types

In [24]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

In [25]:
short_points.dtype

torch.int16

In [26]:
double_points = torch.zeros(10, 2).double()
short_points = torch.zeros(10, 2).short()
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

In [27]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)
double_points.dtype, short_points.dtype

(torch.float64, torch.int16)

## Under the hood: `torch.Storage`

In [28]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [29]:
points_storage = points.storage()
points_storage[0]

4.0

In [30]:
points.storage()[1]

1.0

In [31]:
points_storage[0] = 2.0
points

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])

## Tensor metadata: size, offset, stride

In [36]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]

print(second_point.storage_offset(), second_point.size(), second_point.stride())
print(points.stride())

2 torch.Size([2]) (1,)
(2, 1)


In [39]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0

print(points)

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])


### Cloning a tensor

In [40]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0

print(points)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])


### Transposing a tensor

In [41]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)

points_t = points.t()
print(points_t)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([[4., 5., 2.],
        [1., 3., 1.]])


In [42]:
# same storage
print(id(points.storage()) == id(points_t.storage()))

True


In [43]:
points.stride(), points_t.stride()

((2, 1), (1, 2))

## Making a tensor contiguous

In [44]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()

print(points_t)

tensor([[4., 5., 2.],
        [1., 3., 1.]])


In [45]:
print(points_t.storage())

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]


In [46]:
print(points_t.stride(), points_t.is_contiguous())

(1, 2) False


In [48]:
points_t_cont = points_t.contiguous()
print(points_t_cont)

tensor([[4., 5., 2.],
        [1., 3., 1.]])


In [50]:
id(points_t_cont.storage()) == id(points_t.storage())

True

In [51]:
print(points_t_cont.stride())

(3, 1)


In [52]:
print(points_t_cont.storage())

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]


## Moving tensors to the GPU

In [55]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device="cuda")
print(points_gpu)

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]], device='cuda:0')


## NumPy interoperability

In [61]:
numpy_array = np.ones((4, 3))
print(numpy_array, numpy_array.dtype)

[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] float64


In [69]:
numpy_tensor = torch.from_numpy(numpy_array)
print(numpy_tensor)

print(numpy_array.__array_interface__['data'])

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
(94580442628320, False)


In [70]:
print(numpy_tensor.data_ptr())

94580442628320


In [76]:
numpy_tensor = numpy_tensor.to(dtype=torch.float32)
print(numpy_tensor.data_ptr())

94579654230592


In [72]:
numpy_array.dtype

dtype('float64')

In [73]:
print(numpy_array.__array_interface__['data'])

(94580442628320, False)


In [77]:
numpy_tensor.element_size()

4

In [75]:
numpy_tensor.dtype

torch.float64

## Serializing tensors

In [79]:
torch.save(points, "../data/dlwpt/ch3/ourpoints.t")

### HDF5

In [80]:
import h5py

f = h5py.File("../data/dlwpt/ch3/ourpoints.hdf5", "w")
dset = f.create_dataset("coords", data=points.numpy())
f.close()

In [85]:
f = h5py.File("../data/dlwpt/ch3/ourpoints.hdf5", "r")
print(f.keys())
dset = f["coords"]
last_points = dset[-2:]
print(type(last_points))

<KeysViewHDF5 ['coords']>
<class 'numpy.ndarray'>


## Exercises

### Exercise 1

In [88]:
a = torch.tensor(range(9))
# predictions
# size: 9
# offset: 0
# stride: (1,)
# dtype: int64

print(a.size())
print(a.storage_offset())
print(a.stride())
print(a.dtype)

torch.Size([9])
0
(1,)
torch.int64


In [90]:
b = a.view(3, 3)
print(id(a.storage()), id(b.storage()))
print(b.size(), b.storage_offset(), b.stride())

# view modifies size and stride

140195384094672 140195384094672
torch.Size([3, 3]) 0 (3, 1)


In [92]:
c = b[1:, 1:]

# predictions
# size: (2, 2)
# offset: 4
# stride: (2, 1) WRONG: (3, 1) (think about the underlying storage, not the Tensor size)
print(c.size(), c.storage_offset(), c.stride())

torch.Size([2, 2]) 4 (3, 1)


### Exercise 2

In [96]:
sqrt_a = torch.sqrt(a)
print(sqrt_a)
print(a.sqrt())

tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])
tensor([0.0000, 1.0000, 1.4142, 1.7321, 2.0000, 2.2361, 2.4495, 2.6458, 2.8284])
